# A pair of agents to generate code for data analysis

## Prepare the environment

In [1]:
import tempfile
import shutil
import os
import subprocess

from autogen import ConversableAgent
from autogen.coding import LocalCommandLineCodeExecutor

with open('openai.credential', 'r') as file:
    key = file.read()
MODEL = 'gpt-4o'

## Create a temporary folder to run the code
*Question:* Why not the the work directory, or sub-folder in it?

In [2]:
temp_dir = tempfile.TemporaryDirectory()
print(temp_dir.name)

subprocess.run(["cp", "population.csv", temp_dir.name+"/"])

os.listdir(temp_dir.name)

/tmp/tmpi4w4piz5


['population.csv']

## Code executor. A proxy of command line, no LLM

In [3]:
executor = LocalCommandLineCodeExecutor(
    timeout=10,  # Timeout for each code execution in seconds.
    work_dir=temp_dir.name,  # Use the temporary directory to store the code files.
)

# Create an agent with code executor configuration.
code_executor = ConversableAgent(
    "code_executor_agent",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config={"executor": executor},  # Use the local command line code executor.
    human_input_mode="ALWAYS",  # Always take human input for this agent for safety.
    is_termination_msg=lambda msg: "terminate" in msg["content"].lower(),
)

## Data scientist - the code writer

In [4]:
data_scientist_system_message = """You are a data scientist, and you can solve problems by writing python code. 
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
Reply 'TERMINATE' in the end when everything is done.
Some of the tasks are about analyzing a csv file. If you don't know the structure of the csv file, you may provide code to the users to extract the csv structure, and use the feedback from the users to generate the csv analysis code. 
"""

data_scientist = ConversableAgent(
    "data_scientist_agent",
    system_message = data_scientist_system_message,
    llm_config = {"config_list": [{"model": MODEL, "api_key": key}]},
    code_execution_config=False,  # Turn off code execution for this agent.
)

### Now give the task

In [6]:
chat_result = code_executor.initiate_chat(
    data_scientist,
    message="There is a csv file called population.csv, can you please find out the number of newborns each year for Norway, and store it in a new file result.csv",
)
os.listdir(temp_dir.name)

code_executor_agent (to data_scientist_agent):

There is a csv file called population.csv, can you please find out the number of newborns each year for Norway, and store it in a new file result.csv

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
data_scientist_agent (to code_executor_agent):

Sure, let's break down the task into a few steps:

1. Load the `population.csv` file and explore its structure to understand which columns contain the necessary data.
2. Extract the number of newborns each year for Norway.
3. Store the extracted data in a new file called `result.csv`.

Here's the Python script to perform these steps:

```python
# filename: analyze_population.py
import pandas as pd

# Step 1: Load the CSV file and explore its structure
population_data = pd.read_csv('population.csv')
print(population_data.head())

# Step 2: Filter the data for Norway and the relevant newborns column
# Assuming the newborns column is nam

Provide feedback to data_scientist_agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
code_executor_agent (to data_scientist_agent):

exitcode: 1 (execution failed)
Code output: Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/pandas/core/indexes/base.py", line 3805, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7081, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7089, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'Country'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/tmp/tmpi4w4piz5/analyze_population.py", line 11, 

Provide feedback to data_scientist_agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
code_executor_agent (to data_scientist_agent):

exitcode: 0 (execution succeeded)
Code output: Index(['Country name', 'Year', 'Population',
       'Population of children under the age of 1',
       'Population of children under the age of 5',
       'Population of children under the age of 15',
       'Population under the age of 25', 'Population aged 15 to 64 years',
       'Population older than 15 years', 'Population older than 18 years',
       'Population at age 1', 'Population aged 1 to 4 years',
       'Population aged 5 to 9 years', 'Population aged 10 to 14 years',
       'Population aged 15 to 19 years', 'Population aged 20 to 29 years',
       'Population aged 30 to 39 years', 'Population aged 40 to 49 years',
       'Population aged 50 to 59 years', 'Population aged 60 to 69 years',
       'Population aged 70 to 79 years', 'Population aged 80 to 

Provide feedback to data_scientist_agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
code_executor_agent (to data_scientist_agent):

exitcode: 0 (execution succeeded)
Code output: Number of newborns each year for Norway has been saved to result.csv


--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
data_scientist_agent (to code_executor_agent):

Great! The script has executed successfully, and the number of newborns each year for Norway has been saved to `result.csv`. The task is now completed.

TERMINATE

--------------------------------------------------------------------------------


Provide feedback to data_scientist_agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.


['result.csv',
 'population.csv',
 'determine_columns.py',
 'analyze_population.py']

In [8]:
import pandas as pd
pd.read_csv(temp_dir.name + "/result.csv")

,Year,Newborns
0,1950,61151.0
1,1951,60184.0
2,1952,60279.0
3,1953,61516.0
4,1954,61642.0
...,...,...
67,2017,58155.0
68,2018,56184.0
69,2019,55115.0
70,2020,53986.0
